In [2]:
import pandas as pd

# 加载数据


使用openpyxl引擎

In [4]:
df = pd.read_excel('data.xlsx', engine='openpyxl')
df

,客户是否流失,过去一年进店次数（Total no internal）,全网-过去一年进店次数（Total no internal）,车系,车型,车龄,车龄（天）,是否NEV/PHEV,里程,单位里程消费金额,...,全网历史进店次数（机油保养）,全网历史进店次数（internal）,全网历史进店消费（Total no internal）,全网历史进店消费（客户付费）,全网历史进店消费（保险）,全网历史进店消费（保修）,全网历史进店消费（Total）,全网历史进店消费（BSI）,全网历史进店消费（机油保养）,全网历史进店消费（internal）
0,N,1.0,1.0,X1,X1 20LI,3,1095,N,15599,0.42,...,NaN,1,6549.17,NaN,6549.17,NaN,6549.17,NaN,NaN,0.00
1,Y,0.0,2.0,3 Series,320I A,13,4745,N,100236,1.03,...,12.0,10,107439.66,45864.20,60918.57,656.89,108840.54,NaN,4406.94,1400.88
2,N,0.0,0.0,3 Series,320I A,7,2555,N,32989,0.75,...,1.0,0,24695.27,1139.73,18304.09,5251.45,24695.27,NaN,700.09,NaN
3,N,1.0,1.0,5 Series,523I A,10,3650,N,115782,0.71,...,10.0,6,81849.41,59816.16,21176.39,856.86,83028.25,NaN,7191.39,1178.84
4,Y,0.0,0.0,X1,X1 20LI,3,1095,N,9398,0.03,...,1.0,0,824.97,537.87,NaN,287.10,824.97,NaN,729.04,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33874,NaN,0.0,0.0,X1,X20I A,10,3650,N,86310,0.62,...,9.0,5,53491.36,10463.18,39806.59,3221.59,55607.31,NaN,3706.52,2115.95
33875,NaN,0.0,1.0,3 Series,320I A,2,730,N,7519,0.05,...,1.0,1,350.90,0.00,NaN,350.90,1331.07,NaN,816.47,980.17
33876,NaN,1.0,1.0,X3,X3 20I A,7,2555,N,34567,0.06,...,6.0,5,2051.15,1122.21,NaN,928.94,5470.58,NaN,2996.20,3419.43
33877,NaN,0.0,2.0,3 Series,330I A,4,1460,N,24145,0.36,...,3.0,3,18488.42,2042.36,15601.96,844.10,21120.40,NaN,2161.57,2631.98


## 处理字段


处理名义变量

In [28]:
data = df.drop(columns=[
  '车系', '车型', '是否NEV/PHEV', '最后一次进店经销商是否与本店相同', '最后一次进店车牌号',
  '保修起始日期','最后一次全网进店日期', '本店最后一次进店日期',
  '最后一次进店是否做了客户付费项目',
  '最后一次进店是否做了保险项目',
  '最后一次进店是否做了保修项目',
  '最后一次进店是否做了BSI项目',
  '最后一次进店是否做了机油保养项目',
  '最后一次进店是否做了内部项目',
]).fillna(0)[0:999]
data

,客户是否流失,过去一年进店次数（Total no internal）,全网-过去一年进店次数（Total no internal）,车龄,车龄（天）,里程,单位里程消费金额,单位时间消费金额,非保养清洗类消费,距离下一次续保天数,...,全网历史进店次数（机油保养）,全网历史进店次数（internal）,全网历史进店消费（Total no internal）,全网历史进店消费（客户付费）,全网历史进店消费（保险）,全网历史进店消费（保修）,全网历史进店消费（Total）,全网历史进店消费（BSI）,全网历史进店消费（机油保养）,全网历史进店消费（internal）
0,N,1.0,1.0,3,1095,15599,0.42,2183.06,0.00,211.0,...,0.0,1,6549.17,0.00,6549.17,0.00,6549.17,0.0,0.00,0.00
1,Y,0.0,2.0,13,4745,100236,1.03,7952.19,12867.26,211.0,...,12.0,10,107439.66,45864.20,60918.57,656.89,108840.54,0.0,4406.94,1400.88
2,N,0.0,0.0,7,2555,32989,0.75,3527.90,700.09,96.0,...,1.0,0,24695.27,1139.73,18304.09,5251.45,24695.27,0.0,700.09,0.00
3,N,1.0,1.0,10,3650,115782,0.71,8184.94,14631.20,115.0,...,10.0,6,81849.41,59816.16,21176.39,856.86,83028.25,0.0,7191.39,1178.84
4,Y,0.0,0.0,3,1095,9398,0.03,95.70,0.00,331.0,...,1.0,0,824.97,537.87,0.00,287.10,824.97,0.0,729.04,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,N,1.0,1.0,3,1095,21409,0.16,1120.00,494.56,345.0,...,2.0,1,3359.99,2913.39,0.00,446.60,3800.72,0.0,980.86,440.73
995,N,1.0,1.0,13,4745,303630,0.70,16402.57,24605.79,90.0,...,25.0,25,213233.40,74378.48,112179.81,26675.11,232347.56,0.0,10520.85,19114.16
996,N,1.0,1.0,4,1460,38959,0.41,4031.12,317.90,163.0,...,2.0,1,16124.47,1593.06,14531.41,0.00,16124.47,0.0,1265.64,0.00
997,Y,1.0,2.0,6,2190,57708,0.05,498.57,1346.18,124.0,...,6.0,8,5389.29,2397.84,2672.45,319.00,18307.99,0.0,2178.39,12918.70


### x


数据的参数

In [31]:
X = data.drop(columns=['客户是否流失'])
X

,过去一年进店次数（Total no internal）,全网-过去一年进店次数（Total no internal）,车龄,车龄（天）,里程,单位里程消费金额,单位时间消费金额,非保养清洗类消费,距离下一次续保天数,售价维度得分,...,全网历史进店次数（机油保养）,全网历史进店次数（internal）,全网历史进店消费（Total no internal）,全网历史进店消费（客户付费）,全网历史进店消费（保险）,全网历史进店消费（保修）,全网历史进店消费（Total）,全网历史进店消费（BSI）,全网历史进店消费（机油保养）,全网历史进店消费（internal）
0,1.0,1.0,3,1095,15599,0.42,2183.06,0.00,211.0,1.0,...,0.0,1,6549.17,0.00,6549.17,0.00,6549.17,0.0,0.00,0.00
1,0.0,2.0,13,4745,100236,1.03,7952.19,12867.26,211.0,0.0,...,12.0,10,107439.66,45864.20,60918.57,656.89,108840.54,0.0,4406.94,1400.88
2,0.0,0.0,7,2555,32989,0.75,3527.90,700.09,96.0,4.0,...,1.0,0,24695.27,1139.73,18304.09,5251.45,24695.27,0.0,700.09,0.00
3,1.0,1.0,10,3650,115782,0.71,8184.94,14631.20,115.0,3.0,...,10.0,6,81849.41,59816.16,21176.39,856.86,83028.25,0.0,7191.39,1178.84
4,0.0,0.0,3,1095,9398,0.03,95.70,0.00,331.0,1.0,...,1.0,0,824.97,537.87,0.00,287.10,824.97,0.0,729.04,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,1.0,1.0,3,1095,21409,0.16,1120.00,494.56,345.0,3.0,...,2.0,1,3359.99,2913.39,0.00,446.60,3800.72,0.0,980.86,440.73
995,1.0,1.0,13,4745,303630,0.70,16402.57,24605.79,90.0,0.0,...,25.0,25,213233.40,74378.48,112179.81,26675.11,232347.56,0.0,10520.85,19114.16
996,1.0,1.0,4,1460,38959,0.41,4031.12,317.90,163.0,1.0,...,2.0,1,16124.47,1593.06,14531.41,0.00,16124.47,0.0,1265.64,0.00
997,1.0,2.0,6,2190,57708,0.05,498.57,1346.18,124.0,2.0,...,6.0,8,5389.29,2397.84,2672.45,319.00,18307.99,0.0,2178.39,12918.70


### y


数据标签

In [32]:
y = data['客户是否流失'].replace({'N':0, 'Y':1})
y

0      0
1      1
2      0
3      0
4      1
      ..
994    0
995    0
996    0
997    1
998    1
Name: 客户是否流失, Length: 999, dtype: int64

## 构建模型


svm模型


按照默认数据开始训练

In [20]:
from sklearn import svm

In [33]:
clf = svm.SVC()
clf.fit(X.values, y.values)
clf

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

## 预测


训练好模型之后，用1000条数据做预测

In [48]:
from unittest import result


testdf = df.drop(columns=[
  '客户是否流失',
  '车系', '车型', '是否NEV/PHEV', '最后一次进店经销商是否与本店相同', '最后一次进店车牌号',
  '保修起始日期','最后一次全网进店日期', '本店最后一次进店日期',
  '最后一次进店是否做了客户付费项目',
  '最后一次进店是否做了保险项目',
  '最后一次进店是否做了保修项目',
  '最后一次进店是否做了BSI项目',
  '最后一次进店是否做了机油保养项目',
  '最后一次进店是否做了内部项目',
]).fillna(0)[999:2000]
res = clf.predict(testdf.values)
testdf['预测'] = res
testdf

,过去一年进店次数（Total no internal）,全网-过去一年进店次数（Total no internal）,车龄,车龄（天）,里程,单位里程消费金额,单位时间消费金额,非保养清洗类消费,距离下一次续保天数,售价维度得分,...,全网历史进店次数（internal）,全网历史进店消费（Total no internal）,全网历史进店消费（客户付费）,全网历史进店消费（保险）,全网历史进店消费（保修）,全网历史进店消费（Total）,全网历史进店消费（BSI）,全网历史进店消费（机油保养）,全网历史进店消费（internal）,预测
999,0.0,1.0,3,1095,21334,0.01,95.70,0.00,253.0,1.0,...,2,32990.41,4901.23,27802.08,287.10,33643.10,0.0,936.88,652.69,1
1000,1.0,1.0,3,1095,51759,0.03,570.36,791.36,131.0,3.0,...,4,1711.08,260.30,0.00,1450.78,4748.34,0.0,2546.07,3037.26,0
1001,1.0,1.0,5,1825,23402,0.37,1738.97,2137.81,208.0,3.0,...,4,8694.87,6954.91,0.00,1739.96,14314.29,0.0,2166.62,5619.42,0
1002,1.0,1.0,3,1095,24353,3.46,28110.61,5859.71,6.0,4.0,...,1,84331.82,2858.19,71238.86,10234.77,84331.82,0.0,4454.16,0.00,0
1003,1.0,1.0,4,1460,32767,0.77,6321.45,489.14,308.0,2.0,...,3,25285.79,3175.90,21164.24,945.65,26362.27,0.0,810.99,1076.48,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.0,0.0,12,4380,217009,0.00,60.69,106.35,110.0,4.0,...,0,728.27,728.27,0.00,0.00,728.27,0.0,613.67,0.00,0
1996,0.0,0.0,4,1460,30195,0.01,47.85,0.00,101.0,2.0,...,0,9210.45,0.00,7964.89,1245.56,9210.45,0.0,0.00,0.00,0
1997,1.0,1.0,7,2555,61067,0.51,4488.04,0.00,222.0,4.0,...,3,58064.64,5326.92,40083.63,12654.09,58064.64,0.0,2591.27,0.00,0
1998,0.0,0.0,10,3650,84913,0.00,30.63,314.04,300.0,2.0,...,1,306.32,306.32,0.00,0.00,595.02,0.0,288.70,288.70,0


## 输出文件


输出文件到predication.xlsx

In [49]:
testdf.to_excel('predication.xlsx')